# Time Syncing Data

In [11]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import datetime

In [18]:
## Important Variables

behav_start_frame = 932
behav_end_frame = 27190
behav_start_unix = 1675462020.79345

mini_start_frame = 0
mini_end_frame = 9028
mini_start_unix = 1675462016784/1000

miniscope_frames = -1
behavior_frames = -1

# Behavior @ 30 hz

Time = 1675462020.79345

Behavioral Start: 932

Behavioral End = 27190

Behavioral Frame Duration = 27190 - 932 = 26258

Behavioral Time Duration = 26258 / 30 frames = 875.266666667

Behavior Frames = 30 frames / sec = 0.03 ms

1 second / 30 frames

= 0.0333 sec/frame

932/30 = 31.0667 seconds

Behavior New Zero = 31.0667 seconds

Behavior New Zero Frame = 932

In [2]:
# Behavior Dictionary

b = {
    "start_frame" : -1,
    "end_frame" : -1,
    "frame_duration" : -1,
    "start_unix" : -1,
    "time_duration" : -1
} 

def init_behavior(b, start_frame, end_frame, start_unix):
    # Initialize behavior object based on start/end frame number and start unix time
    b['start_unix'] = start_unix
    b['start_frame'] = start_frame
    b['end_frame'] = end_frame
    b['frame_duration'] = end_frame - start_frame
    b['time_duration'] = behavior_frame_to_time(end_frame - start_frame)
    return b

def behavior_frame_to_time(frame):
    # Convert frame number * (1 sec / 30 frames) = time (s)
    return frame / 30

def behavior_time_to_frame(time):
    # Convert time duration to number of frames
    return time * 30

# behavior = init_behavior(b, behav_start_frame, behav_end_frame, behav_start_unix)    
# print("Behavior = ", behavior)

Behavior =  {'start_frame': 932, 'end_frame': 27190, 'frame_duration': 26258, 'start_unix': 1675462020.79345, 'time_duration': 875.2666666666667}


# Miniscope @ 10 hz

Time = 1675462016784

Miniscope Frames = 10 frames / sec = 0.01 ms

1 sec / 10 frames

= 0.1 sec/frame

Miniscope New Zero = 31.0667 (s) - t_delta = 27.05721650278015 s

Mini New Zero Frame = 270.5721650278015 = 271

Miniscope End Time = 1675462926068

Miniscope End Frames = 9028

In [3]:
# Miniscope Dictionary

# mini_start_frame = 0
# mini_end_frame = 9028
# mini_start_unix = 1675462016784/1000

m = {
    "start_frame" : -1,
    "end_frame" : -1,
    "frame_duration" : -1,
    "start_unix" : -1,
    "time_duration" : -1
} 

def init_miniscope(m, start_frame, end_frame, start_unix):
    # Initialize miniscope object based on start/end frame number and start unix time
    m['start_frame'] = start_frame
    m['end_frame'] = end_frame
    m['frame_duration'] = end_frame - start_frame
    m['start_unix'] = start_unix
    m['time_duration'] = mini_frame_to_time(end_frame - start_frame)
    return m

def mini_frame_to_time(frame):
    # Convert frame number * (1 sec / 30 frames) = time (s)
    return frame / 10

def mini_time_to_frame(time):
    # Convert time duration to number of frames
    return time*10

# miniscope = init_miniscope(m, mini_start_frame, mini_end_frame, mini_start_unix)
# print("Miniscope = ", miniscope)

Miniscope =  {'start_frame': 0, 'end_frame': 9028, 'frame_duration': 9028, 'start_unix': 1675462016.784, 'time_duration': 902.8}


In [29]:
# Update Start Times to Align
#TODO: option for when there are multiple regions to remove from mini or behavior

def print_miniscope():
    print("Miniscope Info")
    print(miniscope)
    print("Raw Start = ", datetime.datetime.fromtimestamp(miniscope['start_unix']))
#     datetime.datetime.fromtimestamp(miniscope['start_unix'] + miniscope['time_duration'])
    new_start_unix = miniscope['start_unix'] + mini_frame_to_time(miniscope['start_frame'])
    new_end_unix = new_start_unix + mini_frame_to_time(miniscope['frame_duration'])
    print("Start/End Timestamp = ", datetime.datetime.fromtimestamp(new_start_unix), " through ", datetime.datetime.fromtimestamp(new_end_unix))

def print_behavior():
    print("Behavior Info")
    print(behavior)
    print("Raw Start = ", datetime.datetime.fromtimestamp(behavior['start_unix']))
#     print("Raw End = ", datetime.datetime.fromtimestamp(behavior['start_unix'] + behavior['time_duration']))
    new_start_unix = behavior['start_unix'] + behavior_frame_to_time(behavior['start_frame'])
    new_end_unix = new_start_unix + behavior_frame_to_time(behavior['frame_duration'])
    print("Start/End Timestamp = ", datetime.datetime.fromtimestamp(new_start_unix), " through ", datetime.datetime.fromtimestamp(new_end_unix))

def update_miniscope(m, start_frame=-1, end_frame=-1):
    # Updates start_frame, frame_duration, and time_duration
    if(start_frame != -1):
        m['start_frame'] =  start_frame
    if(end_frame != -1):
        m['end_frame'] =  end_frame
    m['frame_duration'] = m['end_frame'] - m['start_frame']
    m['time_duration'] = mini_frame_to_time(m['frame_duration'])
    return m
        
def update_behavior(b, start_frame=-1, end_frame=-1):
    # Updates start_frame, frame_duration, and time_duration
    if(start_frame != -1):
        b['start_frame'] =  start_frame
    if(end_frame != -1):
        b['end_frame'] =  end_frame
    b['frame_duration'] = b['end_frame'] - b['start_frame']
    b['time_duration'] = behavior_frame_to_time(b['frame_duration'])
    return b

def update_start_time(behavior, miniscope):
    # Update both behavior and miniscope
    print("Updating Start Time......")
    
    # Determine which starting point is earlier
    time_delta = behavior['start_unix'] - miniscope['start_unix'] 
    print("Time Delta Between Behavior and Miniscope ON = ", time_delta, "(s)")
    
    if time_delta > 0:
        print("Behavior Starts First")
        # This means Behavior Time Starts First (add to new zero for miniscope)
        offset_duration = behavior_frame_to_time(behavior['start_frame']) + time_delta
        mini_new_start_frame = round(mini_time_to_frame(offset_duration))
#         print("New Miniscope Offset = ", str(offset_duration), "(s) and ", str(mini_new_start_frame), "frames")
        miniscope = update_miniscope(miniscope, mini_new_start_frame, -1)
    elif time_delta < 0:
        print("Miniscope Starts First")
        # This means Behavior Time Starts First (add to new zero for miniscope)
        offset_duration = mini_frame_to_time(miniscope['start_frame']) + time_delta
        behavior_new_start_frame = round(behavior_time_to_frame(offset_duration))
        print("New Behavior Offset = ", str(offset_duration), "(s) and ", str(behavior_new_start_frame), "frames")
        behavior = update_behavior(behavior, behavior_new_start_frame, -1)
    
def update_end_times_get_frame_arrays(behavior, miniscope):
    print("Updating End Time......")
    duration_delta = behavior['time_duration'] - miniscope['time_duration']
    miniscope_frames = np.arange(miniscope['start_frame'], miniscope['end_frame'])
    behavior_frames = np.arange(behavior['start_frame'], behavior['end_frame'], 3)

    if duration_delta > 0:
        # np.size(miniscope['frames']) < np.size(behavior['frames'])
        # miniscope['time_duration'] < behavior['time_duration']
        print("Miniscope Ends First")
        behavior_new_end_frame = np.size(miniscope_frames)
        behavior_frames = behavior_frames[:behavior_new_end_frame]        
        behavior = update_behavior(behavior, -1, behavior_frames[-1])
        # update frame end
    elif duration_delta < 0:
        print("Behavior Ends First")
        miniscope_new_end_frame = np.size(behavior_frames)
        miniscope_frames = miniscope_frames[:miniscope_new_end_frame]
        miniscope = update_miniscope(miniscope, -1, miniscope_frames[-1])

    else:
        print("Frame durations are equal! No end time editing necessary.")
        
    if(np.size(behavior_frames) != np.size(miniscope_frames)):
        print("ERROR: Mismatching Array Lengths")
    print("Array size =", np.size(miniscope_frames))
    
    return miniscope_frames, behavior_frames 
    
# update_start_time(behavior, miniscope)
# print("\n")
# miniscope_frames, behavior_frames = update_end_times(behavior,miniscope)
# print("\n")
# print_miniscope()
# print("\n")
# print_behavior()

In [35]:
# Initalize Behavior and Miniscope Objects
behavior = init_behavior(b, behav_start_frame, behav_end_frame, behav_start_unix)
miniscope = init_miniscope(m, mini_start_frame, mini_end_frame, mini_start_unix)

# Sync Up Start Frames
update_start_time(behavior, miniscope)
# Sync Up End Frames and Get Frame Arrays
miniscope_frames, behavior_frames = update_end_times_get_frame_arrays(behavior,miniscope)

print("\n")
print("~ Results ~")
print_miniscope()
print("\n")
print_behavior()
print("\n")

# Now can use and export miniscope_frames and behavior_frames !!!

print(np.size(miniscope_frames) == np.size(behavior_frames))

# miniscope_frames
# behavior_frames

Updating Start Time......
Time Delta Between Behavior and Miniscope ON =  4.009450197219849 (s)
Behavior Starts First
Updating End Time......
Miniscope Ends First
Array size = 8677


~ Results ~
Miniscope Info
{'start_frame': 351, 'end_frame': 9028, 'frame_duration': 8677, 'start_unix': 1675462016.784, 'time_duration': 867.7}
Raw Start =  2023-02-03 17:06:56.784000
Start/End Timestamp =  2023-02-03 17:07:31.884000  through  2023-02-03 17:21:59.584000


Behavior Info
{'start_frame': 932, 'end_frame': 26960, 'frame_duration': 26028, 'start_unix': 1675462020.79345, 'time_duration': 867.6}
Raw Start =  2023-02-03 17:07:00.793450
Start/End Timestamp =  2023-02-03 17:07:31.860117  through  2023-02-03 17:21:59.460117


True
